In [ ]:
!python -VV

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
import numpy as np
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,f1_score,precision_score,roc_auc_score,recall_score,confusion_matrix


In [ ]:
# # Code to read csv file into Colaboratory:
# !pip install -U -q PyDrive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials
# # Authenticate and create the PyDrive client.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

In [ ]:
# id_train = '1KS3lICblsyrxxuJ0giEoeG57-3p3yPng'
# downloaded_train = drive.CreateFile({'id':id_train}) 
# downloaded_train.GetContentFile('norm_train_data') 
# 2mins

In [ ]:
# id_test = '18YCc7R_5kDn_rxovL06ekCGIhO_U15Ty'
# downloaded_test = drive.CreateFile({'id':id_test}) 
# downloaded_test.GetContentFile('norm_test_data')  
# 38sec

In [ ]:
# with open("norm_train_data", "rb") as fh_train:
#   train_data = pickle.load(fh_train)
# with open("norm_test_data", "rb") as fh_test:
#   test_data = pickle.load(fh_test)

In [5]:
train_data = pd.read_pickle("norm_train_data.uu")
test_data = pd.read_pickle("norm_test_data.uu")

In [6]:
def metrics(test_data, prediction):
  cm = pd.DataFrame(confusion_matrix(test_data, prediction))
  cm.columns = ['Predicted Y=0','Predicted Y=1']
  cm.index = ['True Y=0','True Y=1']
  display(cm)

  accuracy = (cm.iloc[0,0]+cm.iloc[1,1])/(cm.iloc[0,0]+cm.iloc[1,1]+cm.iloc[0,1]+cm.iloc[1,0])
  print('Accuracy: '+str(accuracy))

  # Possible ways to further improve accuracy is to try adjusting the level of significance.

  # Calculate Sensitivity (true positive rate)
  sensitivity = cm.iloc[1, 1]/(cm.iloc[1, 1] + cm.iloc[1, 0])
  print('Sensitivity: '+ str(sensitivity))

  # Calculate Specificity (true negative rate)
  specificity = cm.iloc[0, 0]/(cm.iloc[0, 0] + cm.iloc[0, 1])
  print('Specificity: '+str(specificity))

  #Precision
  precision = cm.iloc[1,1]/(cm.iloc[1,1]+cm.iloc[0,1])
  print('Precision: '+str(precision))

  # AUC
  print('ROC-AUC:',roc_auc_score(test_data, prediction))

  #F1 Score
  print("F1 score:", round(f1_score(test_data, prediction), 4))

# Process data

In [ ]:
print(train_data.shape)
print(test_data.shape)

In [ ]:
train_data.columns

In [ ]:
test_data.columns

In [ ]:
print(train_data.isnull().values.any())
print(test_data.isnull().values.any())

In [8]:
train_data.drop(columns=['parent_comment_sentiment'], inplace = True)

In [ ]:
train_data.head()

In [7]:
def sentiment(x):
    if x == 'neu':
        return 0
    elif x == 'pos':
        return 1
    elif x == 'neg': 
        return -1

train_data['comment_sentiment'] = train_data['comment_sentiment'].apply(sentiment)
test_data['comment_sentiment'] = test_data['comment_sentiment'].apply(sentiment)

In [40]:
train_data.head()

,index,label,comment,author,subreddit,score,ups,downs,date,created_utc,...,clean_parent_comment,lemmatized_clean_comment,clean_lemmatized_parent_comment,vectorized_comment,vectorized_clean_comment,vectorized_parent,vectorized_clean_parent,cosine_similarity_dirty_comments,cosine_similarity_clean_comments,comment_sentiment
0,417033,0,But it could mean a loss,warguy3440,GreenBayPackers,1.0,1.0,0.0,2015-11,2015-02-11 03:44:00,...,We need to take some shots down the field Bein...,But could mean loss,We need take shot field Being way fuck careful...,"[-0.23815727, 0.0650364, 0.274105, 0.47422692,...","[-0.22465198, -0.08523826, 0.22933374, 0.56708...","[0.4913653, -0.86889875, 0.6910116, 0.8758221,...","[0.62158334, -0.9648072, 0.6869791, 1.2267407,...",0.538655,0.552133,-1
1,59081,0,Donald the reality TV star does care about TV ...,rk119,politics,1.0,-1.0,-1.0,2016-10,2016-10-21 15:51:00,...,About 71 million watch final presidential deba...,Donald reality TV star care TV rating,About 71 million watch final presidential deba...,"[-0.06878692, 0.64647627, -0.0010952452, 0.650...","[-0.11144994, 0.4386334, -0.11279607, 0.488941...","[0.041787196, -0.3763887, -0.014451511, 0.9660...","[-0.065756515, -0.22653322, -0.20638588, 0.847...",0.582335,0.579743,1
2,5664,1,me irl,PixelPickaxe,Art,1.0,-1.0,-1.0,2016-11,2016-11-27 18:34:00,...,How much you want to bet that the artist was m...,irl,How much want bet artist make regular tiger fu...,"[-0.10693383, 0.044041224, 0.08560472, 0.07766...","[-0.012015545, 0.13237536, 0.031014236, 0.1258...","[0.3451644, -0.49838397, 0.24964134, 0.5616463...","[0.34551588, -0.63334495, 0.33977893, 0.545531...",0.515965,0.510553,0
3,366838,1,Can't have a discussion on one without the oth...,iownakeytar,badwomensanatomy,55.0,55.0,0.0,2016-03,2016-03-28 16:33:00,...,And of course there are the what about circumc...,Cant discussion one without right,And course circumcision assholes cant thing,"[-0.24483381, -0.35850817, 0.5864438, -0.03253...","[-0.54035413, -0.044480685, 0.5084003, 0.08941...","[-0.2357376, -0.49858338, -0.20146894, 0.44658...","[-0.3542063, -0.47738168, -0.24149749, 0.32565...",0.533188,0.423121,0
4,907940,1,That dude's friend's life is nowhere near as i...,HerkDerpner,rage,1.0,1.0,0.0,2013-03,2013-03-26 02:12:00,...,Maybe he found out that one of his friends lik...,That dudes friend life nowhere near important ...,Maybe find one friend like He doesnt want emba...,"[-0.16459303, 0.44265932, 0.76053095, 0.579429...","[-0.3523571, -0.032461423, 0.2875078, 0.775067...","[-0.42899415, 0.1544112, 1.1739204, 1.6521745,...","[-0.54717034, 0.40431726, 1.1967899, 1.6469092...",0.496283,0.471443,-1


In [ ]:
test_data.head()

In [ ]:
print(test_data.shape)
print(train_data.shape)

# Engineered features

In [ ]:
X_train = train_data[['cosine_similarity_dirty_comments','cosine_similarity_clean_comments','word_count','punctuation_count','has_repeated','exclaim_count','qns_mark_count','ellipses_mark_count','interjection_count','laughter_words_count','capitalized_word_count','partial_capital_word_count','emoticon_count']]
Y_train = train_data['label']

X_test = test_data[['cosine_similarity_dirty_comments','cosine_similarity_clean_comments','word_count','punctuation_count','has_repeated','exclaim_count','qns_mark_count','ellipses_mark_count','interjection_count','laughter_words_count','capitalized_word_count','partial_capital_word_count','emoticon_count']]
Y_test = test_data['label']

In [ ]:
rfc = RandomForestClassifier(max_depth=32)

rfc.fit(X_train, Y_train)

In [ ]:
Y_pred = rfc.predict(X_test)

In [ ]:
Y_test = np.asarray(Y_test.astype(int))

In [ ]:
metrics(Y_test,Y_pred)

# Doc2Vec 


In [9]:
# X_train = train_data[['vectorized_clean_comment']]
Y_train = train_data['label']

# X_test = test_data[['vectorized_clean_comment']]
Y_test = test_data['label']

In [ ]:
print(type(Y_train[0]))
print(type(Y_test[0]))

In [ ]:
# print(type(X_train.iloc[0,:]))
# X_train.iloc[0,:][0]

In [ ]:
# print(type(X_train['vectorized_clean_comment']))
# X_train['vectorized_clean_comment'][0]

# Vectorized clean comments

In [ ]:
X_train_columns = pd.DataFrame(train_data['vectorized_clean_comment'].tolist())

In [ ]:
X_test_columns = pd.DataFrame(test_data['vectorized_clean_comment'].tolist())

In [ ]:
rfc = RandomForestClassifier(max_depth=5)

rfc.fit(X_train_columns, Y_train)

11mins to fit rfc = RandomForestClassifier(max_depth=5)

In [ ]:
Y_pred = rfc.predict(X_test_merged)
Y_test = np.asarray(Y_test.astype(int))
metrics(Y_test,Y_pred)

# Vectorized clean comments + engineered features

In [ ]:
X_train_columns = pd.DataFrame(train_data['vectorized_clean_comment'].tolist())
X_train_merged = X_train_columns.join(train_data[['cosine_similarity_dirty_comments','cosine_similarity_clean_comments','word_count','punctuation_count','has_repeated','exclaim_count','qns_mark_count','ellipses_mark_count','interjection_count','laughter_words_count','capitalized_word_count','partial_capital_word_count','emoticon_count']])

In [ ]:
X_test_columns = pd.DataFrame(test_data['vectorized_clean_comment'].tolist())
X_test_merged = X_test_columns.join(train_data[['cosine_similarity_dirty_comments','cosine_similarity_clean_comments','word_count','punctuation_count','has_repeated','exclaim_count','qns_mark_count','ellipses_mark_count','interjection_count','laughter_words_count','capitalized_word_count','partial_capital_word_count','emoticon_count']])

In [ ]:
rfc = RandomForestClassifier(max_depth=5, max_features="sqrt")

rfc.fit(X_train_merged, Y_train)

In [ ]:
Y_pred = rfc.predict(X_test_merged)
Y_test = np.asarray(Y_test.astype(int))
metrics(Y_test,Y_pred)

# Vectorized dirty comments

In [52]:
X_train_dirty_comments = pd.DataFrame(train_data['vectorized_comment'].tolist())
X_test_dirty_comments = pd.DataFrame(test_data['vectorized_comment'].tolist())

In [ ]:
X_train_dirty_comments

In [53]:
rfc1 = RandomForestClassifier(random_state=42)
rfc1.fit(X_train_dirty_comments, Y_train)

RandomForestClassifier(random_state=42)

In [56]:
Y_pred1 = rfc1.predict(X_test_dirty_comments)
Y_test = np.asarray(Y_test.astype(int))

metrics(Y_test,Y_pred1)

,Predicted Y=0,Predicted Y=1
True Y=0,70822,30133
True Y=1,42700,58511


Accuracy: 0.6397366520582096
Sensitivity: 0.5781090988133701
Specificity: 0.7015204794215244
Precision: 0.6600672352330671
ROC-AUC: 0.6398147891174473
F1 score: 0.6164


In [ ]:
Y_pred_df1 = pd.DataFrame(Y_pred1)
Y_pred_df1.to_pickle("./rf_y_pred1")

In [55]:
np.save("./rf_y_pred1",Y_pred1)

# Vectorized dirty comments + engineered features

In [10]:
X_train_dirty_expanded_comments = pd.DataFrame(train_data['vectorized_comment'].tolist())
X_train_merged = X_train_dirty_expanded_comments.join(train_data[['cosine_similarity_dirty_comments','word_count','punctuation_count','has_repeated','exclaim_count','qns_mark_count','ellipses_mark_count','interjection_count','laughter_words_count','capitalized_word_count','partial_capital_word_count','emoticon_count','comment_sentiment']])

In [11]:
X_test_dirty_expanded_comments = pd.DataFrame(test_data['vectorized_comment'].tolist())
X_test_merged = X_test_dirty_expanded_comments.join(test_data[['cosine_similarity_dirty_comments','word_count','punctuation_count','has_repeated','exclaim_count','qns_mark_count','ellipses_mark_count','interjection_count','laughter_words_count','capitalized_word_count','partial_capital_word_count','emoticon_count','comment_sentiment']])

In [65]:
test_data.head()

,index,label,comment,author,subreddit,score,ups,downs,date,created_utc,...,has_repeated,exclaim_count,qns_mark_count,ellipses_mark_count,interjection_count,laughter_words_count,capitalized_word_count,partial_capital_word_count,emoticon_count,comment_sentiment
0,794682,1,Racist comments are really rare on the interne...,Targash,news,6.0,6.0,0.0,2014-06,11/6/14 16:02,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.000000,0.0,-1
1,1003517,0,"yes, if good taste is what a cock doodle shoul...",superfusion1,sex,8.0,8.0,0.0,2011-01,19/1/11 15:32,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.000000,0.0,1
2,526207,1,Totally not photoshopped...,RB30DETT,funny,11.0,11.0,0.0,2015-12,11/12/15 17:29,...,0.0,0.0,0.0,0.0625,0.0,0.0,0.0,0.000000,0.0,0
3,169751,1,"No, there's too much sexism in those industrie...",randomcombination102,TumblrInAction,50.0,-1.0,-1.0,2016-12,18/12/16 2:17,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.000000,0.0,-1
4,585774,0,Maybe they should take a leaf from the lending...,MemorableYetUnique,unitedkingdom,1.0,1.0,0.0,2015-10,19/10/15 20:15,...,0.0,0.0,0.0,0.0625,0.0,0.0,0.0,0.142857,0.0,0


In [61]:
X_test_merged

,0,1,2,3,4,5,6,7,8,9,...,has_repeated,exclaim_count,qns_mark_count,ellipses_mark_count,interjection_count,laughter_words_count,capitalized_word_count,partial_capital_word_count,emoticon_count,comment_sentiment
0,-0.226046,-0.541148,-0.595608,1.673260,-0.354240,-0.281175,1.068446,0.134239,-0.850743,1.826261,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.000000,0.000000,0.0,-1
1,-0.773762,0.477466,0.137423,0.185248,-0.715173,-0.111886,0.751004,0.353527,0.306729,-0.228083,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.000000,0.000000,0.0,1
2,-0.148992,-0.193661,-0.200846,0.687527,-0.207409,0.046413,-0.015980,-0.109896,-0.030504,0.337357,...,0.0,0.0,0.0,0.0625,0.0,0.0,0.000000,0.000000,0.0,0
3,0.248653,0.740374,0.054494,0.811682,-0.346354,-0.214473,-0.085453,0.560107,-0.513053,0.618158,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.000000,0.000000,0.0,-1
4,0.022283,0.008267,0.743422,0.859418,-1.184237,0.434313,-0.605785,1.456591,-0.882477,0.837036,...,0.0,0.0,0.0,0.0625,0.0,0.0,0.000000,0.142857,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202161,-0.225338,0.475583,0.775991,0.405247,-0.349968,0.060040,0.095935,-0.085481,-0.464258,0.026965,...,0.0,0.0,0.0,0.0625,0.0,0.0,0.000000,0.000000,0.0,1
202162,-0.163642,0.070618,-0.027317,0.301575,-0.105552,-0.343681,0.087989,0.310286,0.154339,0.219632,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.002801,0.000000,0.0,-1
202163,-0.011277,0.092318,0.030089,0.207510,-0.313217,0.112651,0.032029,-0.021094,0.121163,0.012707,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.000000,0.000000,0.0,0
202164,0.170538,-0.183067,0.086313,0.575852,0.060031,-0.278173,-0.114822,0.156644,-0.325417,0.296305,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.000000,0.000000,0.0,0


In [23]:
Y_train = train_data['label']
Y_test = test_data['label']

In [21]:
print(Y_train.shape)
print(Y_test.shape)

(807647,)
(202166,)


In [30]:
# untuned hyperparameters
rfc2 = RandomForestClassifier(random_state=42)

rfc2.fit(X_train_merged, Y_train)

RandomForestClassifier(random_state=42)

In [32]:
Y_pred2 = rfc2.predict(X_test_merged)
Y_test = np.asarray(Y_test.astype(int))
metrics(Y_test,Y_pred2)

,Predicted Y=0,Predicted Y=1
True Y=0,74235,26720
True Y=1,42617,58594


Accuracy: 0.6570293719022982
Sensitivity: 0.5789291677782059
Specificity: 0.735327621217374
Precision: 0.6868040415406615
ROC-AUC: 0.65712839449779
F1 score: 0.6283


In [37]:
%%time
a=3
a

Wall time: 0 ns


3

In [18]:
X_train_merged.iloc[:,99:]

,99,cosine_similarity_dirty_comments,word_count,punctuation_count,has_repeated,exclaim_count,qns_mark_count,ellipses_mark_count,interjection_count,laughter_words_count,capitalized_word_count,partial_capital_word_count,emoticon_count,comment_sentiment
0,0.173141,0.538655,0.002251,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,-1
1,0.126739,0.582335,0.004052,0.000102,0.0,0.0,0.000000,0.0,0.0,0.000000,0.001203,0.0,0.0,1
2,0.309487,0.515965,0.000450,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0
3,0.357538,0.533188,0.004052,0.000306,0.0,0.0,0.111111,0.0,0.0,0.000000,0.000000,0.0,0.0,0
4,1.202282,0.496283,0.011256,0.000510,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
807642,0.101653,0.456833,0.007654,0.000204,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000601,0.0,0.0,0
807643,0.855297,0.513966,0.004953,0.000000,0.0,0.0,0.000000,0.0,0.0,0.016667,0.000000,0.0,0.0,1
807644,-0.175838,0.502867,0.006303,0.000510,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0
807645,0.435818,0.479934,0.007204,0.000102,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,1


In [35]:
# tuned hyperparameters
rfc3 = RandomForestClassifier(n_estimators=150, min_samples_split=5, min_samples_leaf=2, random_state=42)

rfc3.fit(X_train_merged, Y_train)

RandomForestClassifier(min_samples_leaf=2, min_samples_split=5,
                       n_estimators=150, random_state=42)

In [36]:
Y_pred3 = rfc3.predict(X_test_merged)
Y_test = np.asarray(Y_test.astype(int))
metrics(Y_test,Y_pred3)

,Predicted Y=0,Predicted Y=1
True Y=0,74292,26663
True Y=1,40706,60505


Accuracy: 0.6667639464598399
Sensitivity: 0.597810514667378
Specificity: 0.7358922292110346
Precision: 0.6941194016152716
ROC-AUC: 0.6668513719392063
F1 score: 0.6424


In [58]:
rfc4 = RandomForestClassifier(n_estimators=150, min_samples_split=5, min_samples_leaf=2, random_state=42, max_samples=0.2)

rfc4.fit(X_train_merged, Y_train)

RandomForestClassifier(max_samples=0.2, min_samples_leaf=2, min_samples_split=5,
                       n_estimators=150, random_state=42)

In [59]:
Y_pred4 = rfc4.predict(X_test_merged)
Y_test = np.asarray(Y_test.astype(int))
metrics(Y_test,Y_pred4)

,Predicted Y=0,Predicted Y=1
True Y=0,73558,27397
True Y=1,40807,60404


Accuracy: 0.6626336772751106
Sensitivity: 0.5968125994210115
Specificity: 0.7286216631172304
Precision: 0.68796482955775
ROC-AUC: 0.6627171312691209
F1 score: 0.6392


In [57]:
metrics(Y_test,Y_pred1)

,Predicted Y=0,Predicted Y=1
True Y=0,70822,30133
True Y=1,42700,58511


Accuracy: 0.6397366520582096
Sensitivity: 0.5781090988133701
Specificity: 0.7015204794215244
Precision: 0.6600672352330671
ROC-AUC: 0.6398147891174473
F1 score: 0.6164


In [44]:
metrics(Y_test,Y_pred2)

,Predicted Y=0,Predicted Y=1
True Y=0,74235,26720
True Y=1,42617,58594


Accuracy: 0.6570293719022982
Sensitivity: 0.5789291677782059
Specificity: 0.735327621217374
Precision: 0.6868040415406615
ROC-AUC: 0.65712839449779
F1 score: 0.6283


In [47]:
metrics(Y_test,Y_pred3)

,Predicted Y=0,Predicted Y=1
True Y=0,74292,26663
True Y=1,40706,60505


Accuracy: 0.6667639464598399
Sensitivity: 0.597810514667378
Specificity: 0.7358922292110346
Precision: 0.6941194016152716
ROC-AUC: 0.6668513719392063
F1 score: 0.6424


In [48]:
Y_pred3

array([1, 0, 1, ..., 0, 0, 0], dtype=int64)

In [49]:
type(Y_pred2)

numpy.ndarray

In [45]:
Y_pred_df2 = pd.DataFrame(Y_pred2)
Y_pred_df2.to_pickle("./rf_y_pred2")

In [50]:
np.save("./rf_y_pred2",Y_pred2)

In [39]:
Y_pred_df3 = pd.DataFrame(Y_pred3)
Y_pred_df3.to_pickle("./rf_y_pred3")

In [51]:
np.save("./rf_y_pred3",Y_pred3)

# Hyperparameter tuning

In [ ]:
X_train_cv = X_train_merged.sample(frac = 0.1, random_state=1)

In [ ]:
X_train_cv.info()

In [ ]:
X_train_merged.info()

In [ ]:
X_1 = X_train_merged.astype('float16')

In [ ]:
type(X_1)

In [ ]:
X_1.info()

In [ ]:
train_data.shape

In [ ]:
train_data.iloc[:,:]

In [ ]:
test_data.shape

In [ ]:
X_1.head()

In [ ]:
X_train_merged.dtypes

In [ ]:
X_train_merged.head()

In [64]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 150, num = 6)]

min_samples_split = [2, 5, 10]

min_samples_leaf = [1, 2, 4]

random_grid = {'n_estimators': n_estimators,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

print(random_grid)

{'n_estimators': [100, 110, 120, 130, 140, 150], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}


In [ ]:
# 54 possible combinations
# choose 27 random comninations to run, choose the combination for the best performing model afterwards
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 27, cv = 3, verbose=10, random_state=42, n_jobs=-2)

In [ ]:
rf_random.fit(X_1, Y_train)

In [ ]:
rf_random.best_params_

# **TFIDF**

# clean comments + cosine sim  + all created features

In [ ]:
data_one = data.drop(['author','subreddit','score','ups','downs','parent_comment',
                      'comment','lemmatized_comment','lemmatized_parent_comment','clean_parent_comment',
                      'clean_lemmatized_parent_comment','vectorized_comment',
                      'vectorized_clean_comment',	'vectorized_parent',	'vectorized_clean_parent','date',
                      'created_utc','lemmatized_clean_comment','cosine_similarity_dirty_comments'], axis = 1)
data_one

In [ ]:
def sent(x):
    if x == 'neu':
        return 0
    elif x == 'pos':
        return 1
    elif x == 'neg': 
        return -1

data_one['comment_sentiment'] = data_one['comment_sentiment'].apply(sent)
data_one['parent_comment_sentiment'] = data_one['parent_comment_sentiment'].apply(sent)
data_one

In [ ]:
 #split
X_train, X_test, y_train, y_test = train_test_split(data_one.drop('label', axis = 1), data_one.label, 
                                                    stratify=data_one.label, 
                                                   random_state=42, 
                                                   test_size=0.2, shuffle=True)

X_train

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
#TODO: tune
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(max_depth=5)

tfidf_1 = TfidfVectorizer()
clmn = ColumnTransformer([("tfidf_1", tfidf_1, "clean_comment")
                         ],
                         remainder="passthrough")

pipe = Pipeline([
                  ('tfidf', clmn),
                  ('classify', model)
                ])



In [ ]:
pipe.fit(X_train,y_train)

In [ ]:
def metrics(test_data, prediction):
  cm = pd.DataFrame(confusion_matrix(test_data, prediction))
  cm.columns = ['Predicted Y=0','Predicted Y=1']
  cm.index = ['True Y=0','True Y=1']
  display(cm)

  accuracy = (cm.iloc[0,0]+cm.iloc[1,1])/(cm.iloc[0,0]+cm.iloc[1,1]+cm.iloc[0,1]+cm.iloc[1,0])
  print('Accuracy: '+str(accuracy))

  # Possible ways to further improve accuracy is to try adjusting the level of significance.

  # Calculate Sensitivity (true positive rate)
  sensitivity = cm.iloc[1, 1]/(cm.iloc[1, 1] + cm.iloc[1, 0])
  print('Sensitivity: '+ str(sensitivity))

  # Calculate Specificity (true negative rate)
  specificity = cm.iloc[0, 0]/(cm.iloc[0, 0] + cm.iloc[0, 1])
  print('Specificity: '+str(specificity))

  #Precision
  precision = cm.iloc[1,1]/(cm.iloc[1,1]+cm.iloc[0,1])
  print('Precision: '+str(precision))

  # AUC
  print('ROC-AUC:',roc_auc_score(test_data, prediction))

  #F1 Score
  print("F1 score:", round(f1_score(test_data, prediction), 4))

In [ ]:
y_pred = pipe.predict(X_test)
metrics(y_test,y_pred)

# dirty comments + cosine sim  + all created features

In [ ]:
data_two = data.drop(['author','subreddit','score','ups','downs','parent_comment',
                      'clean_comment','lemmatized_comment','lemmatized_parent_comment','clean_parent_comment',
                      'clean_lemmatized_parent_comment','vectorized_comment',
                      'vectorized_clean_comment',	'vectorized_parent',	'vectorized_clean_parent','date',
                      'created_utc','lemmatized_clean_comment','cosine_similarity_clean_comments'], axis = 1)

In [ ]:
def sent(x):
    if x == 'neu':
        return 0
    elif x == 'pos':
        return 1
    elif x == 'neg': 
        return -1

data_two['comment_sentiment'] = data_two['comment_sentiment'].apply(sent)
data_two['parent_comment_sentiment'] = data_two['parent_comment_sentiment'].apply(sent)


In [ ]:
 #split
X_train, X_test, y_train, y_test = train_test_split(data_two.drop('label', axis = 1), data_two.label, 
                                                    stratify=data_two.label, 
                                                   random_state=42, 
                                                   test_size=0.2, shuffle=True)

X_train

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
#TODO: tune
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(max_depth=5)

tfidf_1 = TfidfVectorizer()
clmn = ColumnTransformer([("tfidf_1", tfidf_1, "comment")
                         ],
                         remainder="passthrough")

pipe = Pipeline([
                  ('tfidf', clmn),
                  ('classify', model)
                ])



In [ ]:
pipe.fit(X_train,y_train)

In [ ]:
y_pred = pipe.predict(X_test)
metrics(y_test,y_pred)

# dirty comments and cosine sim dirty

In [ ]:
data_three = data[['label','comment','cosine_similarity_dirty_comments']]
data_three.head()

In [ ]:
#CHANGE THIS for different feature subset

 #split
X_train, X_test, y_train, y_test = train_test_split(data_three.drop('label', axis = 1), data_three.label, 
                                                    stratify=data_three.label, 
                                                   random_state=42, 
                                                   test_size=0.2, shuffle=True)

X_train

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score,f1_score,precision_score,roc_auc_score,recall_score,confusion_matrix

model = RandomForestClassifier(max_depth=5)

tfidf_1 = TfidfVectorizer()
clmn = ColumnTransformer([("tfidf_1", tfidf_1, "comment")
                    ],
                    remainder="passthrough")
pipe = Pipeline([
              ('tfidf', clmn),
              ('classify', model)
            ])
pipe.fit(X_train, y_train)

In [ ]:
y_pred = pipe.predict(X_test)
metrics(y_test,y_pred)

# clean comments and cosine sim clean

In [ ]:
data_four = data[['label','clean_comment','cosine_similarity_clean_comments']]
data_four.head()

In [ ]:
#CHANGE THIS for different feature subset

 #split
X_train, X_test, y_train, y_test = train_test_split(data_four.drop('label', axis = 1), data_four.label, 
                                                    stratify=data_four.label, 
                                                   random_state=42, 
                                                   test_size=0.2, shuffle=True)

X_train

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score,f1_score,precision_score,roc_auc_score,recall_score,confusion_matrix

model = RandomForestClassifier(max_depth=5)

tfidf_1 = TfidfVectorizer()
clmn = ColumnTransformer([("tfidf_1", tfidf_1, "clean_comment")
                    ],
                    remainder="passthrough")
pipe = Pipeline([
              ('tfidf', clmn),
              ('classify', model)
            ])
pipe.fit(X_train, y_train)

In [ ]:
y_pred = pipe.predict(X_test)
metrics(y_test,y_pred)